In [3]:
import selenium
from selenium import webdriver
import time
import pandas as pd
import re

In [4]:
def scrollPageToBottomAndFindPostLinks():
    allPosts = []
    
    prevHeight = 0
    newHeight = 1

    while prevHeight != newHeight:
        prevHeight = int(driver.execute_script("return document.body.scrollHeight;"))
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(1)
        allPosts.append(findPostLinks(driver))
        newHeight = int(driver.execute_script("return document.body.scrollHeight;"))
        
    return allPosts; 

def findPostLinks(driver):
    posts = []
    src = driver.page_source
    src_splitted = src.split("</div>")

    for entry in src_splitted:
        postRegex = re.findall("(<a href=\")(.*)(\?taken-by=\w*\">)", entry)
        if len(postRegex) > 0:
            posts.append(postRegex[0][1])
    return posts

def extractPostData():
    return entry

In [5]:
if __name__ == "__main__":
    driver = webdriver.Chrome()
    driver.get("https://www.instagram.com/antlerofficial")
    allPosts = scrollPageToBottomAndFindPostLinks()
    
    #Concatenate all lists in one
    allPosts = sum(allPosts, [])
    #Remove dupes from posts
    allPosts = list(set(allPosts))
    allPosts = ["https://www.instagram.com" + x for x in allPosts]
    print("Amount of links to posts scraped: " + str(len(allPosts)))

In [22]:
postCode = []
for link in allPosts:
    driver.get(link)
    time.sleep(1)
    postCode.append(driver.page_source)

KeyboardInterrupt: 

In [41]:
article = driver.find_element_by_tag_name("article")
txt = article.text
postContent = txt.split("\n")
author = postContent[0]
likes = int(postContent[5].replace(" likes", ""))
post = postContent[6][len(author):]

In [65]:
comments = article.find_element_by_tag_name("li")

In [63]:
usr = comments.find_element_by_tag_name("a")
usr.text

AttributeError: 'list' object has no attribute 'find_element_by_tag_name'

In [66]:
comment = comments.find_elements_by_tag_name("span")

In [69]:
for com in comment:
    print(com.text)

Perfectly combining #style with practicality, our Prospero #suitcase range will add punch to any jet-setting look.
Perfectly combining
with practicality, our Prospero
range will add punch to any jet-setting look.


In [61]:
comment.f

TypeError: 'WebElement' object is not iterable

In [12]:
allPosts[1]

'https://www.instagram.com/p/BDA9L9IqKdq/'

In [8]:
len(allPosts)

694

In [99]:
content = {"link" : "", "author" : "", "post" : "", "likes": 99, "date": "dd-mm-yyyy", "comment": "", "author": ""}

In [65]:
import re
links = re.findall("^(<a href=\")(.*)(\?taken-by=\w*\">).*$", "<a href=\"/p/ptQVgVKKZh/?taken-by=blablaofficial\">")
links

[('<a href="', '/p/ptQVgVKKZh/', '?taken-by=blablaofficial">')]

In [69]:
links[0][1]

'/p/ptQVgVKKZh/'

In [39]:
links = re.finditer("^(<a href=\")(.*)(\?taken-by=\w*\">).*$", src)
links.iter()


for match in regex.finditer(page):
    phrase = match.group(0)
    print("A phrase was found at position", match.start())
    print(repr(phrase))

In [34]:
p = re.compile("^(<a href=\")(.*)(\?taken-by=\w*\">).*$")
print('The pattern is "%s"'%(p.pattern))
for hit in p.finditer(src):
    print(hit.group())
print("Now, look at result of findall")
print(p.findall(src))

NameError: name 're' is not defined